# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boromo,11.7500,-2.9333,31.75,56,62,2.81,BF,1728658530
1,1,hermanus,-34.4187,19.2345,19.89,75,83,3.63,ZA,1728658454
2,2,lebu,-37.6167,-73.6500,12.08,87,100,3.27,CL,1728658532
3,3,nambucca heads,-30.6500,153.0000,15.75,87,41,2.20,AU,1728658533
4,4,gitega,-3.5000,29.9500,19.14,77,86,3.23,BI,1728658535


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                       frame_width=700, frame_height=500,
                                        size = "Humidity", scale=1,color="City", alpha = 0.5

)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filter_df = city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]< 27) &
                             (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"] == 0) ]
filter_df

# Drop any rows with null values
# YOUR CODE HERE

filter_df.dropna(how="any")



# Display sample data
# YOUR CODE HERE
filter_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,el galpon,-25.3807,-64.6526,26.02,41,0,2.34,AR,1728658604
106,106,la manga del mar menor,37.6413,-0.7165,24.04,64,0,3.09,ES,1728658450
108,108,san patricio,28.0170,-97.5169,22.97,50,0,2.89,US,1728658655
149,149,tura,25.5198,90.2201,22.67,90,0,1.42,IN,1728658704
166,166,holualoa,19.6228,-155.9522,24.37,56,0,2.06,US,1728658724


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
clean_df = filter_df.copy()
hotel_df = clean_df.drop(columns=["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"])
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = " "



# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
64,el galpon,AR,-25.3807,-64.6526,41,
106,la manga del mar menor,ES,37.6413,-0.7165,64,
108,san patricio,US,28.0170,-97.5169,50,
149,tura,IN,25.5198,90.2201,90,
166,holualoa,US,19.6228,-155.9522,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = { "categories" : categories, 
   "limit":20,
   "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = hotel_df.loc[index, "Lat"]
    Lon = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{Lon},{Lat},{radius}")
    params["bias"] = (f"proximity:{Lon},{Lat}")

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
el galpon - nearest hotel: No hotel found
la manga del mar menor - nearest hotel: Hotel Entremares
san patricio - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
holualoa - nearest hotel: Kona Hotel
ardestan - nearest hotel: Tourism Hotel
manvel - nearest hotel: No hotel found
dalbandin - nearest hotel: SF Al-dawood Restaurant
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
concepcion del bermejo - nearest hotel: No hotel found
defuniak springs - nearest hotel: No hotel found
ascension - nearest hotel: Cajun Country Inn
blythe - nearest hotel: Quality Inn
sangin - nearest hotel: No hotel found
yigo village - nearest hotel: No hotel found
crane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
64,el galpon,AR,-25.3807,-64.6526,41,No hotel found
106,la manga del mar menor,ES,37.6413,-0.7165,64,Hotel Entremares
108,san patricio,US,28.0170,-97.5169,50,No hotel found
149,tura,IN,25.5198,90.2201,90,Hotel Polo Orchid
166,holualoa,US,19.6228,-155.9522,56,Kona Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                       frame_width=700, frame_height=500,
                                       size = "Humidity", scale=1,color="City", alpha = 0.50,
                                       hover_cols=["Hotel Name", "Country"]

)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)